# ⌬──── Setup ────⌬

In [ ]:
# @title ♻️ For Google Drive Authentication
#@markdown <br><center><img src='https://user-images.githubusercontent.com/125879861/255377947-6ac19c35-dbbd-4a9b-bc0e-c603de81c533.png' height="70" alt="Gdrive-logo"/></center>
#@markdown <center><h2><font color=yellow><b>Mount Google Drive</b></h2></center><br>

#@markdown <br><h3><font color=cyan><b>🐬 To Mount Google Drive in Colab</b></h3>

MODE = "Mount"  # @param ["Mount", "Unmount", "Nothing"]
MOUNT_TO = "/content/drive"  # @param ["/content/drive"]

from google.colab import drive

drive.mount._DEBUG = False

if MODE == "Mount":
    drive.mount(MOUNT_TO, force_remount=True)
elif MODE == "Unmount":
    try:
        drive.flush_and_unmount()
    except ValueError:
        pass

In [ ]:
# @title 🔁 Clone Repo & Install Semua Dependensi

# 🚀 Clone repo GitHub
!rm -rf video_downloader
!git clone https://github.com/lIlSkaSkaSkalIl/video_downloader

# 📦 Install Python requirements dari file
!pip install -r video_downloader/requirements.txt

# ✅ Tambahkan repo ke sys.path agar bisa import uploader.*
import sys

repo_path = "/content/video_downloader"
if repo_path not in sys.path:
    sys.path.append(repo_path)

# Downloader

In [ ]:
# @title 🍪 Upload Cookies
# @markdown - Upload file cookies (Untuk download twitter yang membutuhkan auth)
from google.colab import files
import os
from utils.directories import prepare_directories

# 📁 Siapkan direktori
dirs = prepare_directories()
cookies_dir = dirs["cookies"]
cookies_path = os.path.join(cookies_dir, "cookies.txt")

print("📤 Silakan upload file cookies.txt (hasil dari Kiwi Browser)...")
uploaded = files.upload()

# Temukan file .txt yang diupload
cookies_file = None
for name in uploaded.keys():
    if name.endswith(".txt"):
        cookies_file = name
        break

# Simpan ke cookies_dir dengan nama cookies.txt
if cookies_file and os.path.exists(cookies_file):
    os.makedirs(cookies_dir, exist_ok=True)
    if cookies_file != "cookies.txt":
        os.rename(cookies_file, "cookies.txt")

    os.replace("cookies.txt", cookies_path)
    print(f"📁 File disimpan sebagai → {cookies_path}")

    # Tampilkan 3 baris awal isi file
    print("📄 Contoh isi cookies.txt:")
    with open(cookies_path, "r", encoding="utf-8") as f:
        for i in range(3):
            line = f.readline()
            if line:
                print("  ", line.strip())
else:
    print("❌ File .txt tidak valid ditemukan atau gagal upload.")

In [ ]:
# @title 🎞️ Unduh Video Twitter
# @markdown - Masukkan URL tweet (single-video / multi-video)

# ✅ Install yt-dlp (jika belum)
!pip install -U yt-dlp > /dev/null

tweet_url = ""  # @param {type: "string"}

# 📁 Setup direktori dan path repo
import sys
repo_path = "/content/video_downloader"
if repo_path not in sys.path:
    sys.path.append(repo_path)

from downloader.twitter import (
    simulate_metadata,
    download_tweet_video
)
from utils.directories import prepare_directories
from utils.messages import build_twitter_summary

# ✅ Setup folder
dirs = prepare_directories()

# 🔍 Simulasi metadata (optional)
simulate_metadata(tweet_url)

# 📥 Mulai download
tweet_id, use_cookies, video_dir, duration = download_tweet_video(tweet_url, base_dir=dirs["base"])

# 📦 Status akhir
import glob, os
downloaded = glob.glob(os.path.join(video_dir, "*_video.*"))
print(build_twitter_summary(tweet_url, tweet_id, use_cookies, downloaded, video_dir, duration))

In [ ]:
# @title 🚀 Download Video M3U8/TS
# @markdown - URL bisa file .m3u8 langsung atau video streaming .ts
video_url = ""  # @param {type:"string"}
output_name = ""  # @param {type:"string"}

# Tambahkan repo path
import sys
repo_path = "/content/video_downloader"
if repo_path not in sys.path:
    sys.path.append(repo_path)

from downloader.m3u8 import download_m3u8_video

download_m3u8_video(video_url, output_name)

# Uploader

In [ ]:
# @title 🎯 Ambil Metadata Video
video_dir = ""  # @param {type:"string"}

import os
import sys

# ✅ Tambahkan path ke repo lokal (hanya jika belum)
sys.path.append("/content/video_downloader")

# ✅ Siapkan folder output
from utils.directories import prepare_directories
from uploader.tools.metadata.processor import process_all_videos  # ⬅️ Panggil dari processor.py

# 🔧 Siapkan direktori output (meta, thumb, log)
dirs = prepare_directories("/content/media_toolkit")
meta_dir = dirs["meta"]
thumb_dir = dirs["thumb"]
log_txt_path = os.path.join(dirs["logs"], "log.txt")
log_json_path = os.path.join(dirs["logs"], "log.json")

# 🚀 Jalankan proses metadata extractor
process_all_videos(
    video_dir=video_dir,
    metadata_dir=meta_dir,
    thumbnail_dir=thumb_dir,
    log_txt_path=log_txt_path,
    log_json_path=log_json_path
)

In [ ]:
# @title ✈️ Upload ke Telegram

import os
import sys
import asyncio

# ✅ Tambahkan path ke repo GitHub
sys.path.append("/content/video_downloader")

# ✅ Import fungsi direktori & upload
from utils.directories import prepare_directories
from uploader.core.batch_upload import batch_upload

# 🔧 Siapkan direktori kerja
dirs = prepare_directories("/content/media_toolkit")
meta_dir = dirs["meta"]
logs_dir = dirs["logs"]
log_txt = os.path.join(logs_dir, "log.txt")
log_json = os.path.join(logs_dir, "log.json")

# ⚙️ Parameter Colab
CHANNEL_ID = 1234  # @param {type:"number"}
CHAT_ID = 1234         # @param {type:"number"}
API_ID = 1234            # @param {type:"number"}
API_HASH = ""  # @param {type:"string"}
BOT_TOKEN = ""  # @param {type:"string"}

# 🚀 Jalankan proses upload
await batch_upload(
    metadata_dir=meta_dir,
    log_txt_path=log_txt,
    log_json_path=log_json,
    CHAT_ID=CHAT_ID,
    CHANNEL_ID=CHANNEL_ID,
    API_ID=API_ID,
    API_HASH=API_HASH,
    BOT_TOKEN=BOT_TOKEN
)

# Utilities

In [ ]:
# @title 🧹 Hapus Isi Folder
folder_path = ""  # @param {type:"string"}

import os
import shutil

def format_size(bytes):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes < 1024:
            return f"{bytes:.2f} {unit}"
        bytes /= 1024
    return f"{bytes:.2f} PB"

def print_summary(folder):
    total_files = 0
    total_size = 0

    for root, dirs, files in os.walk(folder):
        for file in files:
            total_files += 1
            try:
                filepath = os.path.join(root, file)
                total_size += os.path.getsize(filepath)
            except:
                pass

    print("Ringkasan File:")
    print("╭📁 Jumlah file      :", total_files)
    print("├💾 Total ukuran     :", format_size(total_size))
    print("╰🎯 Lokasi Folder    :", folder)
    return len(os.listdir(folder))

def delete_folder_contents(folder):
    if not os.path.exists(folder):
        print(f"❌ Folder tidak ditemukan: {folder}")
        return

    if not os.path.isdir(folder):
        print(f"⚠️ Bukan folder: {folder}")
        return

    all_items = os.listdir(folder)
    total_items = len(all_items)
    if total_items == 0:
        print("📂 Folder sudah kosong.")
        return

    print_summary(folder)
    print()

    count = 1
    for item in all_items:
        item_path = os.path.join(folder, item)
        try:
            print(f"🗑️ [{count}/{total_items}] Menghapus {item} ... ", end="")
            if os.path.isfile(item_path) or os.path.islink(item_path):
                os.unlink(item_path)
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
            print("✅")
        except Exception as e:
            print(f"❌ ({e})")
        count += 1

    print("\n🧹 Proses selesai. Semua isi folder telah dihapus.")

delete_folder_contents(folder_path)

In [ ]:
# @title 🎬 [Insert] Embed Subtitle (.srt) → Output MKV
# @markdown - Masukkan path lengkap video dan subtitle (.srt)
video_input_path = ""  # @param {type:"string"}
subtitle_path = ""  # @param {type:"string"}

import os
import subprocess

# Ambil nama asli video tanpa ekstensi
video_basename = os.path.splitext(os.path.basename(video_input_path))[0]

# Output video (mkv)
output_video_path = os.path.join(output_dir, f"{video_basename}_with_subtitles.mkv")

# Jalankan ffmpeg
cmd = [
    "ffmpeg",
    "-i", video_input_path,
    "-i", subtitle_path,
    "-c", "copy",        # tanpa re-encode
    "-c:s", "srt",       # jaga format subtitle .srt
    "-map", "0",
    "-map", "1",
    output_video_path
]

print("🎞️ Menyisipkan subtitle ke video (.mkv, softsub)...")
subprocess.run(cmd, check=True)
print(f"✅ Video akhir disimpan di: {output_video_path}")

In [ ]:
# @title 📤 [Manual] Cek & Ekstrak Subtitle Softsub dari Video
# @markdown - Masukkan path video yang ingin dicek
video_with_sub_path = ""  # @param {type:"string"}

import subprocess, os, json

# Lokasi output subtitle
subtitle_extracted_path = os.path.join(output_dir, "extracted_subtitle.srt")

# 1️⃣ Cek dengan ffprobe apakah subtitle stream tersedia
probe_cmd = [
    "ffprobe",
    "-v", "error",
    "-select_streams", "s",
    "-show_entries", "stream=index:stream_tags=language",
    "-of", "json",
    video_with_sub_path
]

probe_result = subprocess.run(probe_cmd, capture_output=True, text=True)

# Parse hasil JSON
streams = json.loads(probe_result.stdout).get("streams", [])

if not streams:
    print("⚠️ Video tidak memiliki stream subtitle (softsub).")
else:
    print(f"✅ Ditemukan {len(streams)} subtitle stream. Mengekstrak stream pertama...")

    # 2️⃣ Jalankan ffmpeg untuk ekstrak subtitle pertama
    extract_cmd = [
        "ffmpeg",
        "-y",
        "-i", video_with_sub_path,
        "-map", "0:s:0",
        subtitle_extracted_path
    ]
    result = subprocess.run(extract_cmd, capture_output=True, text=True)

    if result.returncode == 0 and os.path.exists(subtitle_extracted_path):
        print(f"✅ Subtitle berhasil diekstrak ke: {subtitle_extracted_path}")
    else:
        print("❌ Gagal mengekstrak subtitle.")
        print("📄 FFmpeg log:")
        print(result.stderr)

# Kompres/Ekstrak File

In [ ]:

# @title 📦 Setup Project
import os
if not os.path.exists("/content/compress_decompress_archive"):
    !git clone https://github.com/lIlSkaSkaSkalIl/compress_decompress_archive.git

import sys
sys.path.append("/content/compress_decompress_archive")
print("✅ Repo siap digunakan.")

In [ ]:
# @title ⚙️ Kompresi / Dekompresi

from tools.archive_tool import run_tool

# 👉 Pilih metode dan path
metode = "untar"  # @param ["zip", "unzip", "rar", "unrar", "7z", "un7z", "tar", "untar"]
tar_method = "none"  # @param ["gz", "xz", "none"]
input_path = ""  # @param {type:"string"}
output_path = ""  # @param {type:"string"}

# 🚀 Jalankan
if metode == "tar":
    run_tool(metode, input_path, output_path, tar_method=tar_method)
else:
    run_tool(metode, input_path, output_path)